# SQL Connection using `.env`
* Credentials are excluded from the notebook file
* Relies on a separate `.env` file in an absolute or relative path to the notebook file

#### Details

Example contents of the `.env` file:
``` env
# Development settings
DOMAIN=example.org
ADMIN_EMAIL=admin@${DOMAIN}
ROOT_URL=${DOMAIN}/app
```

Jupyter Notebook references the contents of the `.env` file by:
* Importing packages using magics, `%load_ext dotenv`, `%dotenv`
* `import os`
* Assign `.env` variables into a `global_var`. 
  * Example: `email = os.getenv("ADMIN_EMAIL")`
  * Output : `admin@example.org`
* `.env` file needs to be excluded from version control by adding the following to your `.gitignore`:

####

## `.env` variables

In [1]:
import os
import requests
import ibm_db
import ibm_db_dbi
import pandas as pd
# magics for import statements
%load_ext dotenv
%dotenv

In [2]:
# Replace the placeholder values with the actuals for your Db2 Service Credentials
DSN_DRIVER = os.getenv("dsn_driver")
DSN_DB = os.getenv("dsn_database")
DSN_HOSTNAME = os.getenv("dsn_hostname")
DSN_PORT = os.getenv("dsn_port")
DSN_PROTOCOL = os.getenv("dsn_protocol")
DSN_UID = os.getenv("dsn_uid")
DSN_PWD = os.getenv("dsn_pwd")
DSN_SECURITY = os.getenv("dsn_security")

## Connection using `ibm_db` library

In [13]:
# Create database connection
# DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(DSN_DRIVER, DSN_DB, DSN_HOSTNAME, DSN_PORT, DSN_PROTOCOL, DSN_UID, DSN_PWD, DSN_SECURITY)
#                    format(dsn_driver, dsn_db, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd, dsn_security)
# print(dsn)

try:
    conn = ibm_db.connect(dsn, "", "")
    print("Connected to database: ", DSN_DB, "as user: ", DSN_UID, "on host: ", DSN_HOSTNAME)
except:
    print("Unable to connect: ", ibm_db.conn_errormsg())

Connected to database:  BLUDB as user:  mkr43631 on host:  fbd88901-ebdb-4a4f-a32e-9822b9fb237b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud


In [14]:
# connection for pandas
pconn = ibm_db_dbi.Connection(conn)
# query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INTERNATIONAL_STUDENT_TEST_SCORES"

# retrieve the query results into a pandas dataframe
pdf = pd.read_sql(selectQuery, pconn)

In [15]:
# print just the LNAME for first row in the pandas data frame
pdf.head()

,COUNTRY,FIRST_NAME,LAST_NAME,TEST_SCORE
0,United States,Marshall,Bernadot,54
1,Ghana,Celinda,Malkin,51
2,Ukraine,Guillermo,Furze,53
3,Greece,Aharon,Tunnow,48
4,Russia,Bail,Goodwin,46


In [75]:
try:
    ibm_db.close(conn)
except Exception:
    print('Connection is not active')

Connection is not active


## SQL Magics

In [21]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [69]:
# sql magic
conn_str = 'ibm_db_sa://'+DSN_UID+":"+DSN_PWD+"@"+DSN_HOSTNAME+":"+DSN_PORT+"/"+DSN_DB.lower()+"?security="+DSN_SECURITY
# print(var1==conn_str)
# print(var1)
# print(conn_str)
%sql $conn_str

In [70]:
country = "Canada"
%sql select * from INTERNATIONAL_STUDENT_TEST_SCORES where country = :country

 * ibm_db_sa://mkr43631:***@fbd88901-ebdb-4a4f-a32e-9822b9fb237b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32731/bludb
Done.


country,first_name,last_name,test_score
Canada,Cristionna,Wadmore,46
Canada,Wilhelm,Deeprose,54
Canada,Carma,Schule,49


In [71]:
# list of current database connections and their properties
conn_list = %sql l / --connections
print(conn_list)

{'ibm_db_sa://mkr43631:***@fbd88901-ebdb-4a4f-a32e-9822b9fb237b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32731/bludb': <sql.connection.Connection object at 0x0000017CE91177C0>}
